# Fine tune a checkpoint of MedSAM on point prompted Data

Given a path to a MedSAM checkpoint, we want to fine tune it on pre-processed data
(subject to modifications specified by the paper and the transformation script). This will
be done initially on an anatomy-specific level.

## Argparse Setup

In [1]:
# Imports
import re
import os
import cv2
import sys
import json
import torch
import monai
import random
import argparse
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
from time import time, sleep
from datetime import datetime
from matplotlib import pyplot as plt
from segment_anything import sam_model_registry
from torch.utils.data import Dataset, DataLoader

# Add the setup_data_vars function as we will need it to find the directory for the training data.
dir1 = os.path.abspath(os.path.join(os.path.abspath(''), '..', '..'))
if not dir1 in sys.path: sys.path.append(dir1)

from utils.environment import setup_data_vars
setup_data_vars()

In [ ]:
parser = argparse.ArgumentParser()

# Inspired by orginal code from the MedSAM/extensions/point_prompt

# 1. Add the anatomy on which we will fine-tune
parser.add_argument(
    '--anatomy',
    type=str,
    help='Anatomy on which to fine-tune the model. Note: this is case sensitive, please capitalize the first letter and accronyms such as CTVn or CTVp.',
    required=True
)

# 2. Path to the MedSAM checkpoint
parser.add_argument(
    '--checkpoint',
    type=str,
    help='Path to the checkpoint of the model to fine-tune',
    required=True
)

# 3. Path where we will be saving the checkpoints of the fine-tuned model
parser.add_argument(
    '--save_dir',
    type=str,
    help='Directory where the fine-tuned model will be saved',
    required=True
)

# 4. Add the source directory for the data
parser.add_argument(
    '--img_dir',
    type=str,
    help='Directory containing the images for the slices of the anatomy',
    required=False,
)

# 5. Add the source directory for the gts
parser.add_argument(
    '--gt_dir',
    type=str,
    help='Directory containing the ground truth masks for the slices of the anatomy',
    required=False
)

# 6. Number of epochs for the fine-tuning
parser.add_argument(
    '--epochs',
    type=int,
    help='Number of epochs for the fine-tuning',
    required=False,
    default=300
)

# 7. Batch size for the fine-tuning
parser.add_argument(
    '--batch_size',
    type=int,
    help='Batch size for the fine-tuning',
    required=False,
    default=4
)

# 8. Learning rate for the fine-tuning
parser.add_argument(
    '--lr',
    type=float,
    help='Learning rate for the fine-tuning',
    required=False,
    default=0.00005
)

# 9. Number of workers for the data loader
parser.add_argument(
    '--num_workers',
    type=int,
    help='Number of workers for the data loader',
    required=False,
    default=4
)

parser.add_argument(
    '--weight_decay',
    type=float,
    help='Weight decay for the optimizer',
    required=False,
    default=0.01
)

# 11. Resume checkpoint
parser.add_argument(
    '--resume',
    type=bool,
    help='Whether to resume training using the latest checkpoint in the save_dir',
    required=False,
    default=True
)

_StoreAction(option_strings=['--resume'], dest='resume', nargs=None, const=None, default=True, type=<class 'bool'>, choices=None, required=False, help='Whether to resume training using the latest checkpoint in the save_dir', metavar=None)

In [ ]:
# args = parser.parse_args()
# Suppose for now we get the following set of required arguments:
args = parser.parse_args([
    '--anatomy', 'CTVn',
    '--checkpoint', os.path.join(os.environ['PROJECT_DIR'], 'models', 'MedSAM', 'work_dir', 'MedSAM', 'medsam_vit_b.pth'),
    '--save_dir', os.path.join(os.environ['MedSAM_finetuned'], 'CTVn')
])

## Set up the vars

In [ ]:
anatomy = args.anatomy
checkpoint_path = args.checkpoint
save_dir = args.save_dir
img_dir = args.img_dir
gt_dir = args.gt_dir
epochs = args.epochs
batch_size = args.batch_size
lr = args.lr
num_workers = args.num_workers
weight_decay = args.weight_decay
resume = args.resume

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
if img_dir is None:
    img_dir = os.path.join(os.environ['MedSAM_preprocessed'], 'imgs')
if gt_dir is None:
    gt_dir = os.path.join(os.environ['MedSAM_preprocessed'], 'gts', anatomy)

In [ ]:
seed = 42

torch.cuda.empty_cache()
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
image_id_from_file_name_regex = r'.*_(\d+).*'
slice_id_from_file_name_regex = r'.*-(\d+).*'

## Set up Dataset class

In [ ]:
# Adapted Dataset class from ../2_no_finetuning/MEDSAM_helper_functions.py
class SAM_Dataset(Dataset):
    """A torch dataset for delivering slices of any axis to a medsam model."""

    def __init__(self, img_path, gt_path, id_split, data_aug=False):
        """
        Args:
            img_path (string): Path to the directory containing the images
            gt_path (string): Path to the directory containing the ground truth masks
            id_split (list): List of image ids to include in the dataset
        """

        self.root_img_path = img_path
        self.root_gt_path = gt_path
        self.id_split = id_split
        self.data_aug = data_aug
        
        # Assume that axese 0 1 and 2 have been processed.
        filter_fn = lambda x : x.endswith('.npy') and int(re.search(image_id_from_file_name_regex, x).group(1)) in id_split
        self.axis0_imgs = list(filter(filter_fn, os.listdir(os.path.join(gt_path, 'axis0'))))
        self.axis1_imgs = list(filter(filter_fn, os.listdir(os.path.join(gt_path, 'axis1'))))
        self.axis2_imgs = list(filter(filter_fn, os.listdir(os.path.join(gt_path, 'axis2'))))

    def __len__(self):
        return len(self.axis0_imgs) + len(self.axis1_imgs) + len(self.axis2_imgs)

    def __getitem__(self, idx):
        assert 0 <= idx < self.__len__(), f"Index {idx} is out of range for dataset of size {self.__len__()}"

        # Fetch the image and ground truth mask. For safety, we index the items around the
        # ground truth masks, so that if for some reason the images are misaligned we will
        # guarantee that we will fetch the correct image

        if idx < len(self.axis0_imgs):
            axis, gt_name = 0, self.axis0_imgs[idx]
        elif idx < len(self.axis0_imgs) + len(self.axis1_imgs):
            axis, gt_name = 1, self.axis1_imgs[idx - len(self.axis0_imgs)]
        else:
            axis, gt_name = 2, self.axis2_imgs[idx - len(self.axis0_imgs) - len(self.axis1_imgs)]

        image_id = int(re.search(image_id_from_file_name_regex, gt_name).group(1))
        slice_id = int(re.search(slice_id_from_file_name_regex, gt_name).group(1))

        img_name = f'CT_zzAMLART_{image_id:03d}-{slice_id:03d}.npy'
        
        # Load the image and ground truth mask

        img_path = os.path.join(self.root_img_path, f'axis{axis}', img_name)
        gt_path = os.path.join(self.root_gt_path, f'axis{axis}', gt_name)

        img = np.load(img_path, 'r', allow_pickle=True) # (H, W, C)
        gt = np.load(gt_path, 'r', allow_pickle=True) # (H, W, C)

        # Pre-process where necessary

        img = np.transpose(img, (2, 0, 1)) # (C, H, W)
        assert np.max(img) <= 1. and np.min(img) >= 0., 'image should be normalized to [0, 1]'

        # add data augmentation: random fliplr and random flipud
        if self.data_aug:
            if random.random() > 0.5:
                img = np.ascontiguousarray(np.flip(img, axis=-1))
                gt = np.ascontiguousarray(np.flip(gt, axis=-1))
            if random.random() > 0.5:
                img = np.ascontiguousarray(np.flip(img, axis=-2))
                gt = np.ascontiguousarray(np.flip(gt, axis=-2))
        
        gt = np.uint8(gt > 0)
        y_indices, x_indices = np.where(gt > 0)
        x_point = np.random.choice(x_indices)
        y_point = np.random.choice(y_indices)
        coords = np.array([x_point, y_point])

        # The output of the model is 256x256, and it is easier to reason about constricting an image, rather than expanding the output back ot 1024x1024

        gt = cv2.resize(
            gt,
            (256, 256),
            interpolation=cv2.INTER_NEAREST
        )

        return {
            "image": torch.tensor(img).float(),
            "gt2D": torch.tensor(gt[None, :,:]).long(),
            "coords": torch.tensor(coords[None, ...]).float(),
            "image_name": img_name
        }

## Set up Fine-Tuning nn Module

In [ ]:
class MedSAM(nn.Module):
    def __init__(self, 
                image_encoder, 
                mask_decoder,
                prompt_encoder,
                freeze_image_encoder=False,
                ):
        super().__init__()
        self.image_encoder = image_encoder
        self.mask_decoder = mask_decoder
        self.prompt_encoder = prompt_encoder

        # freeze prompt encoder
        for param in self.prompt_encoder.parameters():
            param.requires_grad = False
        
        self.freeze_image_encoder = freeze_image_encoder
        if self.freeze_image_encoder:
            for param in self.image_encoder.parameters():
                param.requires_grad = False

    def forward(self, image, point_prompt):

        # do not compute gradients for pretrained img encoder and prompt encoder
        with torch.no_grad():
            image_embedding = self.image_encoder(image) # (B, 256, 64, 64)
            # not need to convert box to 1024x1024 grid
            # bbox is already in 1024x1024
            sparse_embeddings, dense_embeddings = self.prompt_encoder(
                points=point_prompt,
                boxes=None,
                masks=None,
            )
        low_res_masks, iou_predictions = self.mask_decoder(
            image_embeddings=image_embedding, # (B, 256, 64, 64)
            image_pe=self.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
            sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
            dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
            multimask_output=False,
          ) # (B, 1, 256, 256)

        return low_res_masks

## Setup logger for viewing progress

In [ ]:
class MedSAMLogger(object):
    """The purpose of this class is to log the training process of the MedSAM model in a
    format that can be easily visualized."""

    " - start_epoch - this will start recording the per-batch loss for this new epoch, also records start time"
    
    " - end_epoch - this will end the recording of the per-batch loss for this epoch, also records end time. This will return and store the reduced loss as well as the duration for the epoch."

    def __init__(self, save_dir):
        epoch_stats_template = {
            'train_loss': [],
            'train_dice': [],
            'train_loss_reduced': -1,
            'train_dice_reduced': -1,
            'val_loss': [],
            'val_dice': [],
            'val_loss_reduced': -1,
            'val_dice_reduced': -1,
            'epoch_time': {
                'start': -1,
                'end': -1,
                'duration': 0
            }
        }

        # attempt to create a logger.json file if it doesn't already exist. If it does,
        # assume that we are resuming training
        if not os.path.exists(os.path.join(save_dir, 'logger.json')):
            self._load_logger()
        else:
            self.epoch_data = {
                'epoch': 0,
                'epoch_stats': epoch_stats_template
            }
        


    def log_train(self, loss, dice):
        self.train_loss.append(loss)
        self.train_dice.append(dice)

    def log_val(self, loss, dice):
        self.val_loss.append(loss)
        self.val_dice.append(dice)    

    def start_of_epoch(self):
        """initialize the loss and dice lists for the epoch"""
        self.train_loss = []
        self.train_dice = []

        self.val_loss = []
        self.val_dice = []

    def end_of_epoch(self):
        """calculate the epoch time and the average of the losses and dice scores"""


    def log_epoch_time(self, time):
        self.epoch_time.append(time)


## Main Training Loop

In [ ]:
import logging

logging.basicConfig(
    filename=os.path.join(save_dir, 'MedSAM_training.log'), 
    filemode='w',
    format='[%(levelname)s : %(asctime)s] - %(message)s',
    datefmt='%d-%b-%y %H:%M:%S',
    level=logging.DEBUG
)

# logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

logging.debug('Set up logger! Happy Debugging!')

Set up logger! Happy Debugging!


In [ ]:
class MedSAMTrainer(object):
    def __init__(self
                , anatomy
                , checkpoint_path
                , save_dir
                , image_dir
                , gt_dir
                , epochs
                , batch_size
                , lr
                , num_workers
                , weight_decay
                , resume
                , device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                , ):
        
        self.anatomy = anatomy
        assert self.anatomy in ['CTVn', 'CTVp', 'Bladder', 'Anorectum', 'Uterus', 'Vagina']
        
        self.image_dir = image_dir
        self.gt_dir = gt_dir
        assert os.path.exists(self.image_dir), 'Image Directory doesn\'t exist.'
        assert os.path.exists(self.gt_dir), 'Ground Truth Directory doesn\'t exist for the requested anatomy.'

        self.save_dir = save_dir
        self.checkpoint_path = checkpoint_path
        self.epochs = epochs
        self.batch_size = batch_size
        self.lr = lr
        self.num_workers = num_workers
        self.weight_decay = weight_decay
        self.resume = resume
        self.device = device

        # self.stat_logger = MedSAMLogger(self.save_dir)

    def run_training(self):
        """Main Training Loop"""

        logging.debug('Entered main training loop')
        self.on_train_start()

        for epoch in range(self.start_epoch, self.epochs):
            # <Training>
            self.on_epoch_start(epoch)

            pbar = tqdm(self.train_loader)
            for step, batch in enumerate(pbar):
                dice_loss, ce_loss = self.train_step(step, batch)

                # self.stat_logger.log_train(dice_loss, ce_loss)

                pbar.set_description(f"Epoch {epoch} at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, loss: {dice_loss + ce_loss:.4f}")

            self.on_epoch_end()
            # </Training>

            # # <Validation>
            # with torch.no_grad():
            #     self.medsam_model.eval()
            #     for step, batch in enumerate(self.val_loader):
            #         image = batch["image"].to(self.device)
            #         gt2D = batch["gt2D"].to(self.device)
            #         coords_torch = batch["coords"].to(self.device)
            # # </Validation>

        self.on_train_end()

    def train_step(self, step, batch):
        logging.debug(f'Starting epoch {self.current_epoch} step {step} at time {datetime.now()}')

        # Get data
        logging.debug('getting the data and setting to device')
        image = batch["image"].to(self.device)
        gt2D = batch["gt2D"].to(self.device)
        coords_torch = batch["coords"].to(self.device) # (B, 2)

        logging.debug('Performing forward pass')
        self.optimizer.zero_grad()
        labels_torch = torch.ones(coords_torch.shape[0]).long() # (B,)
        labels_torch = labels_torch.unsqueeze(1) # (B, 1)
        coords_torch, labels_torch = coords_torch.to(self.device), labels_torch.to(self.device)
        point_prompt = (coords_torch, labels_torch)
        medsam_lite_pred = self.medsam_model(image, point_prompt)

        logging.debug('Calculating loss')
        dice_loss = self.dice_loss(medsam_lite_pred, gt2D)
        ce_loss = self.ce_loss(medsam_lite_pred, gt2D.float())

        loss = dice_loss + ce_loss
        logging.debug(f'Loss calculated: {loss.item()}')
        
        logging.debug('Performing step in optimizer and propagating loss backgwards')
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()

        # self.stat_logger.log_train(dice_loss.item(), ce_loss.item())

        return dice_loss.item(), ce_loss.item()
    
    def on_train_end(self):
        logging.debug(f'Reached the end of training! Epochs: {self.epochs}')
        self.save_checkpoint(self.current_epoch, self.epoch_loss, final=True)
        torch.cuda.empty_cache()

    def on_epoch_end(self):
        # epoch_time, epoch_los_reduced = self.stat_logger.end_of_epoch(self.current_epoch)

        self.epoch_end_time = time()
        self.epoch_time.append(self.epoch_end_time - self.epoch_start_time)
        logging.debug(f'Reached the end of epoch {self.current_epoch}, it took {self.epoch_end_time - self.epoch_start_time} seconds')

        epoch_loss_reduced = sum(self.epoch_loss) / len(self.epoch_loss)

        self.save_checkpoint(self.current_epoch, epoch_loss_reduced)

        # Plot the progress

        # self.print_to_log_file('train_loss', np.round(logging.my_fantastic_logging['train_losses'][-1], decimals=4))
        # self.print_to_log_file('val_loss', np.round(logging.my_fantastic_logging['val_losses'][-1], decimals=4))
        # self.print_to_log_file('Pseudo dice', [np.round(i, decimals=4) for i in
        #                                        logging.my_fantastic_logging['dice_per_class_or_region'][-1]])
        # self.print_to_log_file(
        #     f"Epoch time: {np.round(logging.my_fantastic_logging['epoch_end_timestamps'][-1] - logging.my_fantastic_logging['epoch_start_timestamps'][-1], decimals=2)} s")

        # logging.plot_progress_png(self.save_dir)

    def save_checkpoint(self, epoch, epoch_loss_reduced, final=False):
        # If the checkpoint is better than the last, save it as 'checkpoint_best.pth' otherwise, save it as checkpoint_latest.pth

        logging.debug(f'Saving a checkpoint for epoch {epoch}')        

        checkpoint = {
            "model": self.medsam_model.state_dict(),
            "epochs": epoch,
            "optimizer": self.optimizer.state_dict(),
            "loss": epoch_loss_reduced,
            "best_loss": self.best_loss
        }

        if epoch_loss_reduced < self.best_loss:
            logging.debug(f'Found new best checkpoint! Loss: {epoch_loss_reduced:.4f}')
            self.best_loss = epoch_loss_reduced
            torch.save(checkpoint, os.path.join(self.save_dir, 'checkpoint_best.pth'))

        if final:
            logging.debug(f'Saving final version of the checkpoing')
            torch.save(checkpoint, os.path.join(self.save_dir, 'checkpoint_final.pth'))
            os.remove(os.path.join(self.save_dir, 'checkpoint_latest.pth'))
        else:
            logging.debug(f'Saving latest checkpoint path')
            torch.save(checkpoint, os.path.join(self.save_dir, 'checkpoint_latest.pth'))

    def on_train_start(self):
        """Sets up the training environment"""
        logging.debug('starting training environment')

        # empty cuda cache
        torch.cuda.empty_cache()

        # create save_dir if it doesn't exist yet
        os.makedirs(self.save_dir, exist_ok=True)

        # set up logger to a new instance
        # logging = MedSAMLogger()

        # load the previous checkpoint if it exists and resume is True. Otherwise, load
        # the medSAM model from which we train. 
        self._first_run_setup()
        if (not self._maybe_load_checkpoint()):
            self._setup_data_splits()

        # set up the dataloaders
        self._get_dataloaders()

        # set up loss functions
        logging.debug('Setting up loss functions')
        self.dice_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean')
        self.ce_loss = nn.BCEWithLogitsLoss(reduction="mean")

    def on_epoch_start(self, epoch):        
        self.medsam_model.train()
        self.current_epoch = epoch
        self.epoch_start_time = datetime.now()
        self.epoch_loss = [1e10 for _ in range(len(self.train_loader))]

        # self.stat_logger.start_epoch(epoch, epoch_start_time)

        logging.debug(f'>> Starting epoch {epoch} at time {self.epoch_start_time}')

    def _maybe_load_checkpoint(self):
        """
        Populates the variables IF a checkpoint has been found
        - checkpoint
        - medsam_model
        - optimizer
        - start_epoch
        - best_loss
        - training_split
        - validation_split
        """
        logging.debug('Attempting to find and load checkpoint from save directory')
        if not self.resume:
            logging.debug('Resume was set to false, skipping checkpoint loading.')
            return False

        if len([f for f in os.listdir(save_dir) if f == 'checkpoint_latest.pth']) == 0:
            logging.debug('No checkpoint found in the save directory')
            return False
        
        # Load model details
        logging.debug('found and loading checkpoint from checkpoint_latest.pth')
        self.checkpoint = torch.load(os.path.join(save_dir, 'checkpoint_latest.pth'))

        self.medsam_model.load_state_dict(self.checkpoint["model"])
        self.optimizer.load_state_dict(self.checkpoint["optimizer"])
        self.start_epoch = self.checkpoint["epoch"] + 1
        self.best_loss = self.checkpoint["best_loss"]
        logging.debug(f"Loaded checkpoint from epoch {self.start_epoch}, best loss: {self.best_loss:.4f}")

        # Get Data Split
        try:
            self._load_split_from_json()
        except FileNotFoundError as e:
            logging.debug('[WARNING] a checkpoint was found, but a split file was not.')
            logging.debug('          a new data split will be generated with potential training bias towards the validaiton split.')
            self._setup_data_splits()

        return True  

    def _first_run_setup(self):
        logging.debug('running setup for sam model at base checkpoint')
        sam_model = sam_model_registry["vit_b"](checkpoint=self.checkpoint_path)
        self.medsam_model = MedSAM(
            image_encoder = sam_model.image_encoder,
            mask_decoder = sam_model.mask_decoder,
            prompt_encoder = sam_model.prompt_encoder,
            freeze_image_encoder = True
        )
        self.medsam_model = self.medsam_model.to(self.device)

        logging.debug('setting up optimizer for sam model at base checkpoint')
        self.optimizer = optim.AdamW(
            self.medsam_model.mask_decoder.parameters(),
            lr=self.lr,
            betas=(0.9, 0.999),
            eps=1e-08,
            weight_decay=self.weight_decay
        )

        self.start_epoch = 0
        self.best_loss = float('inf')

    def _save_splits_to_json(self, training_image_ids, validation_image_ids):
        logging.debug('Saving data splits to json')
        data = {
            "training_image_ids": list(training_image_ids),
            "validation_image_ids": list(validation_image_ids)
        }
        with open(os.path.join(self.save_dir, 'data_splits.json'), 'w') as json_file:
            json.dump(data, json_file)

    def _load_split_from_json(self):
        logging.debug('Loading the data split from save directory')
        with open(os.path.join(self.save_dir, 'data_splits.json'), 'r') as json_file:
            data = json.load(json_file)
        self.training_split = set(data["training_image_ids"])
        self.validation_split = set(data["validation_image_ids"])

    def _setup_data_splits(self, training_split=None, validation_split=None):
        """Setup the data splits either from a known source or a new setup."""
        logging.debug('Setting up the data splits for training and validation sets.')
        if training_split is not None and validation_split is not None:
            logging.debug('Existing splits found, setting them up.')
            self.training_split = training_split
            self.validation_split = validation_split
            return

        # get the image ids that have been processed. Use gt dir as reference
        axis0_slices = set(map(lambda x : int(re.search(image_id_from_file_name_regex, x).group(1)), os.listdir(os.path.join(self.gt_dir, 'axis0'))))
        axis1_slices = set(map(lambda x : int(re.search(image_id_from_file_name_regex, x).group(1)), os.listdir(os.path.join(self.gt_dir, 'axis1'))))
        axis2_slices = set(map(lambda x : int(re.search(image_id_from_file_name_regex, x).group(1)), os.listdir(os.path.join(self.gt_dir, 'axis2'))))

        if not axis0_slices == axis1_slices == axis2_slices:
            print('[WARNING]: The slices for the anatomy are not consistent across the three axes. Some axese are missing data, please check')

        training_split = 0.8
        validation_split = 1 - training_split

        # Split the data into training and validation
        self.training_image_ids = random.sample(list(axis0_slices), int(len(axis0_slices) * training_split))
        self.validation_image_ids = list(set(axis0_slices) - set(self.training_image_ids))
        assert set.intersection(set(self.training_image_ids), set(self.validation_image_ids)).__len__() == 0, 'Training and Validation sets are not disjoint'

        # Save the splits in a json file
        self._save_splits_to_json(self.training_image_ids, self.validation_image_ids)

    def _get_dataloaders(self):
        logging.debug('Setting up Datasets and DataLoaders using the splits.')
        self.training_dataset = SAM_Dataset(self.image_dir, self.gt_dir, self.training_image_ids)
        self.validation_dataset = SAM_Dataset(self.image_dir, self.gt_dir, self.validation_image_ids)
        
        # Quick check
        assert set(map(lambda x : int(re.search(image_id_from_file_name_regex, x).group(1)), self.validation_dataset.axis0_imgs)) == set(self.validation_image_ids), 'DataSet incorrectly loaded image ids that don\'t match supplied validation set image ids'
        assert set(map(lambda x : int(re.search(image_id_from_file_name_regex, x).group(1)), self.training_dataset.axis0_imgs)) == set(self.training_image_ids), 'DataSet incorrectly loaded image ids that don\'t match supplied validation set image ids'

        self.train_loader = DataLoader(self.training_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
        self.val_loader = DataLoader(self.validation_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

In [ ]:
trainer = MedSAMTrainer(
    anatomy
    , checkpoint_path
    , save_dir
    , img_dir
    , gt_dir
    , epochs
    , batch_size
    , lr
    , num_workers
    , weight_decay
    , resume
)

In [ ]:
trainer.run_training()

Entered main training loop
starting training environment
running setup for sam model at base checkpoint
setting up optimizer for sam model at base checkpoint
Attempting to find and load checkpoint from save directory
No checkpoint found in the save directory
Setting up the data splits for training and validation sets.
Saving data splits to json
Setting up Datasets and DataLoaders using the splits.
Setting up loss functions
>> Starting epoch 0 at time 2024-05-17 19:11:56.097333


  0%|          | 0/6741 [00:00<?, ?it/s]

Starting epoch 0 step 0 at time 2024-05-17 19:11:56.609581
getting the data and setting to device
Performing forward pass
Calculating loss
Loss calculated: 0.9801114201545715
Performing step in optimizer and propagating loss backgwards


Epoch 0 at 2024-05-17 19:11:57, loss: 0.9801:   0%|          | 1/6741 [00:00<1:51:34,  1.01it/s]

Starting epoch 0 step 1 at time 2024-05-17 19:11:57.095254
getting the data and setting to device
Performing forward pass
Calculating loss
Loss calculated: 1.1390385627746582
Performing step in optimizer and propagating loss backgwards


...

Starting epoch 0 step 347 at time 2024-05-17 19:17:17.151492
getting the data and setting to device
Performing forward pass
Calculating loss
Loss calculated: 0.4992715120315552
Performing step in optimizer and propagating loss backgwards


Epoch 0 at 2024-05-17 19:17:17, loss: 0.4993:   5%|▌         | 348/6741 [05:21<1:33:34,  1.14it/s]

Starting epoch 0 step 348 at time 2024-05-17 19:17:17.598327
getting the data and setting to device
Performing forward pass
Calculating loss
Loss calculated: 0.558003842830658
Performing step in optimizer and propagating loss backgwards


Epoch 0 at 2024-05-17 19:17:18, loss: 0.5580:   5%|▌         | 349/6741 [05:23<1:38:49,  1.08it/s]


KeyboardInterrupt: 